Launch interactive version: 👉👉👉 [![Try ``dyce``](https://jupyterlite.readthedocs.io/en/latest/_static/badge.svg)](https://posita.github.io/dyce-notebooks/lab?path=stack-exchange%2Funder-target-over-challenge-206876%2Funder_target_over_challenge.ipynb) 👈👈👈 *[[source](https://github.com/posita/dyce-notebooks/tree/main/notebooks/stack-exchange/under-target-over-challenge-206876)]*

## [``dyce``](https://posita.github.io/dyce/) solution to [“Anydice - Roll a d20 under target number but beat an opposed die”](https://rpg.stackexchange.com/a/206911/71245)

Once viewing this notebook in Jupyter Lab, select ``Run All Cells`` from the ``Run`` menu above.

In [1]:
# Install additional requirements if necessary
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    try:
        import anydyce
    except (ImportError, ModuleNotFoundError):
        # See <https://jupyterlite.readthedocs.io/en/stable/howto/configure/simple_extensions.html#avoid-the-drift-of-versions-between-the-frontend-extension-and-the-python-package>
        requirements = ["ipycanvas==0.13.2", "ipyevents==2.0.1", "ipympl==0.9.4", "ipywidgets==8.1.3", "anydyce==0.4.6"]
        try:
            import piplite ; await piplite.install(requirements, keep_going=True)
            # Work around <https://github.com/jupyterlite/jupyterlite/issues/838>
            import matplotlib.pyplot ; matplotlib.pyplot.clf()
        except ImportError:
            import pip ; pip.main(["install"] + requirements)
    import anydyce

try:
    import showit
except ImportError:
    # Work-around for JupyterLite in non-Chromium browsers
    import js
    import os
    from urllib.parse import urljoin, urlparse, urlunparse
    loc_url = urlparse(js.location.toString())
    ext_root = loc_url.path.find("/extensions/@jupyterlite/")
    if ext_root < 0:
        base_url = urljoin(js.location.toString(), "../files/")
    else:
        loc_url = loc_url._replace(path=loc_url.path[:ext_root])
        base_url = urljoin(urlunparse(loc_url), "files/")
    for path in (
                "stack-exchange/under-target-over-challenge-206876/showit.py",
            ):
        url = urljoin(base_url, path)
        res = await js.fetch(url)
        assert 200 <= res.status < 300
        text = await res.text()
        with open(os.path.basename(path), "w") as f:
            f.write(text)
    import showit

In [2]:
from dyce import H
from dyce.evaluation import HResult, foreach
from showit import showit
from enum import IntEnum, auto

class ResultType(IntEnum):
    IMPOSSIBLE = auto()  # no roll can succeed because target <= challenge die
    TARGET_MISS = auto()  # failure because player die >= target
    CHALLENGE_MISS = auto()  # failure because player die <= challenge die
    HIT = auto()  # success

d4 = H(4)
d6 = H(6)
d8 = H(8)
d10 = H(10)
d12 = H(12)
d20 = H(20)
d6_3 = 3 @ d6
d10_2 = 2 @ d10
d8d12 = d8 + d12

available_player_dice = {
    "d20": d20,
    "3d6": d6_3,
    "2d10": d10_2,
    "d8 + d12": d8d12,
}

available_challenge_dice = {
    "d4": d4,
    "d6": d6,
    "d8": d8,
    "d10": d10,
    "d12": d12,
}
# Include the player dice as well
available_challenge_dice.update(available_player_dice)

def expected_result(
    player_die: H,
    challenge_die: H,
    target: int,
) -> H:
    def _dependent_term(player_die: HResult, challenge_die: HResult):
        if target <= challenge_die.outcome:
            return ResultType.IMPOSSIBLE
        elif player_die.outcome >= target:
            return ResultType.TARGET_MISS
        elif player_die.outcome <= challenge_die.outcome:
            return ResultType.CHALLENGE_MISS
        else:
            return ResultType.HIT

    # Start with zero counts for all possible outcomes
    result_base = H((outcome, 0) for outcome in ResultType)
    # Accumulate those that came up in our calculation
    return result_base.accumulate(foreach(_dependent_term, player_die, challenge_die))

def expected_result_low_fidelity(
    player_die: H,
    challenge_die: H,
    target: int,
) -> H:
    # Build a die that has just the faces that are below the target
    # (or zero where they were at or above)
    player_target_die = H(
        (outcome, count) if outcome < target else (0, count)
        for outcome, count in player_die.items()
    )
    # Then compute a histogram for where those faces are greater than
    # the challenge die
    return player_target_die.gt(challenge_die)

showit(
    available_player_dice,
    available_challenge_dice,
    expected_result,  # or use expected_result_low_fidelity instead
)

Output()